In [1]:
from azureml.core import Workspace, Experiment, Dataset,Datastore

In [9]:
path='C:/Users/satya/Downloads/credit_risk_dataset.csv/credit_risk_dataset.csv'
workspace=Workspace.create(name='automl-K21',resource_group='automl-K21RG',location='eastus2',create_resource_group=True,subscription_id='1db33695-8135-4616-9bb4-9574b401d454',exist_ok=True)

In [10]:
default_ds=workspace.get_default_datastore()
default_ds.upload_files(files=[path],target_path='data/',overwrite=True, show_progress=True)


Uploading an estimated of 1 files
Uploading C:/Users/satya/Downloads/credit_risk_dataset.csv/credit_risk_dataset.csv
Uploaded C:/Users/satya/Downloads/credit_risk_dataset.csv/credit_risk_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_45f8e8a246184c41b50a49d23a5b2bac

In [11]:
dataset_credit=Dataset.Tabular.from_delimited_files(path=(default_ds,'data/credit_risk_dataset.csv'))
dataset_credit=dataset_credit.register(workspace=workspace,name='credit_risk_dataset')
dataset_credit.to_pandas_dataframe()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,True,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,False,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,False,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,False,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,57,53000,MORTGAGE,1.0,PERSONAL,C,5800,13.16,0,0.11,False,30
32577,54,120000,MORTGAGE,4.0,PERSONAL,A,17625,7.49,0,0.15,False,19
32578,65,76000,RENT,3.0,HOMEIMPROVEMENT,B,35000,10.99,1,0.46,False,28
32579,56,150000,MORTGAGE,5.0,PERSONAL,B,15000,11.48,0,0.10,False,26


In [14]:
from azureml.core.compute import AmlCompute,ComputeTarget
compute_name='computeclusterk21'
compute_config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_target=ComputeTarget.create(workspace, compute_name, compute_config)
compute_target.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [28]:
#import automl
from azureml.train.automl import AutoMLConfig

automl_config=AutoMLConfig(
    task='classification',
    primary_metric='AUC_weighted',
    training_data=dataset_credit,
    validation_size=0.25,
    n_cross_validations=5,
    label_column_name='loan_status',
    compute_target=compute_target,
    max_concurrent_iterations=4,
    blocked_models=['RandomForest']
)


In [29]:
#import Run details
experiment=Experiment(workspace,name='automl_Credit_risk')
run_automl=experiment.submit(automl_config,show_output=True)


Submitting remote run.
No run_configuration provided, running on computeclusterk21 with default configuration
Running on remote compute: computeclusterk21


Experiment,Id,Type,Status,Details Page,Docs Page
automl_Credit_risk,AutoML_1bcc32fd-6a25-4fc7-90a8-b694ac9fff57,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing featu

In [39]:
run_automl.get_best_child()

Experiment,Id,Type,Status,Details Page,Docs Page
automl_Credit_risk,AutoML_1bcc32fd-6a25-4fc7-90a8-b694ac9fff57_36,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [32]:
best_model =run_automl.get_children()

In [34]:
for run in run_automl.get_children():
    print(run.id)
    for metric in run.get_metrics():
        print(metric,run.get_metrics(metric))

AutoML_1bcc32fd-6a25-4fc7-90a8-b694ac9fff57_36
norm_macro_recall {'norm_macro_recall': 0.7264754669772577}
recall_score_micro {'recall_score_micro': 0.9364841640068746}
matthews_correlation {'matthews_correlation': 0.8087855538897983}
balanced_accuracy {'balanced_accuracy': 0.863237733488629}
recall_score_weighted {'recall_score_weighted': 0.9364841640068746}
recall_score_macro {'recall_score_macro': 0.863237733488629}
precision_score_micro {'precision_score_micro': 0.9364841640068746}
f1_score_macro {'f1_score_macro': 0.8978530685485981}
accuracy {'accuracy': 0.9364841640068746}
precision_score_macro {'precision_score_macro': 0.950225668001967}
precision_score_weighted {'precision_score_weighted': 0.9386996825812364}
weighted_accuracy {'weighted_accuracy': 0.9746601221699052}
AUC_macro {'AUC_macro': 0.9525952069610228}
AUC_weighted {'AUC_weighted': 0.9525952069610228}
average_precision_score_macro {'average_precision_score_macro': 0.9471562914409825}
log_loss {'log_loss': 0.1828288552

KeyboardInterrupt: 

In [37]:
#best model details
best_run, fitted_model = run_automl.get_output()
print(best_run)



Package:azureml-core, training version:1.48.0, current version:1.43.0
Package:azureml-dataprep, training version:4.8.3, current version:4.0.4
Package:azureml-dataprep-rslex, training version:2.15.1, current version:2.6.3
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.43.0.post2
Package:azureml-interpret, training version:1.48.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.48.0, current version:1.43.0
Package:azureml-telemetry, training version:1.48.0, current version:1.43.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.43.0
Package:azureml-train-core, training version:1.48.0, current version:1.43.0
Package:azureml-train-restclients-hyperdrive, training version:1.48.0, current version:1.43.0
Package:azureml-defaults, training version:1.48.0
Package:azureml-inference-server-http, training version:0.7.7
Package:azureml-mlflow, training version:1.48.0
Package:azureml-responsibleai, training version:

Run(Experiment: automl_Credit_risk,
Id: AutoML_1bcc32fd-6a25-4fc7-90a8-b694ac9fff57_36,
Type: azureml.scriptrun,
Status: Completed)


In [38]:
run_automl.get_best_child(metrics='AUC_weighted')

Experiment,Id,Type,Status,Details Page,Docs Page
automl_Credit_risk,AutoML_1bcc32fd-6a25-4fc7-90a8-b694ac9fff57_36,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation
